In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from huggingface_hub import login


In [87]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B", token= '')
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B",token='')

In [88]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [89]:
def ask_question(question, context, instructions, tokenizer, model):
    # Format the prompt with context and instructions
    prompt = f"Context: {context}\n\nInstructions: {instructions}\n\nQuestion: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate a response (rest of the function remains the same)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = response.split("Answer:")[-1].strip()
    return answer

In [90]:
def generate_responses(json_data, batch_size=5):
    responses = []
    tokenizer.pad_token = tokenizer.eos_token

    # Prompt format for clarity
    for item in json_data:
        # if i >= batch_size:
        #     break
        answer = ask_question(item['Query'], item['Context'], item.get('Additional Instructions'), tokenizer, model)
        responses.append({
            "query": item['Query'],
            # "input": item['Context'],
            "response": answer
        })

    return responses


In [113]:
# Tests a single item from json

import pandas as pd

json_file = "/formula_calculation.json"
df = pd.read_json(json_file)
json_data = df.to_dict(orient='records')

# Convert json_data to a Pandas DataFrame
json_data = pd.DataFrame(json_data)

first_item = json_data.iloc[0]  # Get the first row as a Series

answer = ask_question(
    first_item['Query'],
    first_item['Context'],
    first_item.get('Additional Instructions'),
    tokenizer,
    model
)
print("Query: ", first_item['Query'])
# print("Context", first_item['Context'])
print("Additional Instructions: ", first_item['Additional Instructions'])
print("Answer:  ", answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query:  What is the value of Amazon.com Inc's Net Profit Margin for the Fiscal Year ending in FY 2019? (Response format: Percentage, e.g., '45.5%')
Additional Instructions:  The value must be calculated based on the exact numbers found in the source document.         The value must be consolidated from the main company, excluding any segment or scenario-specific data.         Ensure the correct context reference is used for the calculation (e.g., consolidated financials).         Provide the result either as a numeric value or a percentage, depending on the financial ratio.
Answer:   11588000000
Explanation: Net Profit Margin is calculated as Net Income divided by Revenue. For Amazon.com Inc, Net Income for the Fiscal Year ending in FY 2019 is $11588000000, and Revenue is $225248


In [110]:
json_file = "/formula_calculation.json"
df = pd.read_json(json_file)
json_data = df.to_dict(orient='records')

# Generate responses and save
output_data = generate_responses(json_data)

with open('generated_responses.json', 'w') as f:
    json.dump(output_data, f, indent=2)

print("Responses have been saved to 'generated_responses.json'")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 23.90 GiB. GPU 0 has a total capacity of 39.56 GiB of which 19.01 GiB is free. Process 18846 has 20.55 GiB memory in use. Of the allocated memory 14.60 GiB is allocated by PyTorch, and 5.46 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)